In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Input
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling2D, Activation, Dense, Dropout
from keras.utils import to_categorical
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay

In [2]:
#download the weather images and put them in a directory
path = r'C:\Users\suhas\Documents\careerfoundry\ml\datasets\multiclass-weather'
folders = os.listdir(path)
print(folders)

['Cloudy', 'Rain', 'Shine', 'Sunrise']


In [3]:
# rescale the pictures to 250x250

datagenerator = ImageDataGenerator(rescale=1/255, validation_split=0.2)
train_generator = datagenerator.flow_from_directory(
    path,
    target_size=(250 , 250),
    batch_size=32,
    subset='training',
)

test_generator = datagenerator.flow_from_directory(
    path,
    target_size=(250 , 250),
    batch_size=32,
    subset='validation',
)

Found 901 images belonging to 4 classes.
Found 224 images belonging to 4 classes.


In [4]:
#this is a CNN to categorize the weather images

inputs = Input(shape=(250, 250, 3))
x = Conv2D(32, (5,5), activation = 'relu')(inputs)
x = MaxPooling2D(3,3)(x)
x = Conv2D(64, (5,5) , activation = 'relu')(x)
x = MaxPooling2D((3,3))(x)
x = Conv2D(128 , (5,5) , activation = 'relu')(x)
x = MaxPooling2D(3,3)(x)
x = Flatten()(x)
x = Dropout(0.5, seed = 5)(x)
x = Dense(128 , activation = 'relu')(x)

outputs = Dense(4, activation = 'softmax')(x)

model:Model = Model(inputs=inputs, outputs=outputs)

In [7]:
#compile and run the model
#how many epochs are required to generate an accurate model?

model.compile(optimizer="Adam", loss="mean_squared_error", metrics=["accuracy"])
model.fit(
    train_generator,
    epochs = 5,
    steps_per_epoch = train_generator.n // train_generator.batch_size,
    validation_data = test_generator,
    validation_steps = test_generator.n // test_generator.batch_size,
)

Epoch 1/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 18s 582ms/step - accuracy: 0.7327 - loss: 0.0844 - val_accuracy: 0.8438 - val_loss: 0.0574
Epoch 2/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.8750 - loss: 0.0354 
Epoch 3/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 585ms/step - accuracy: 0.7893 - loss: 0.0701 - val_accuracy: 0.7634 - val_loss: 0.0833
Epoch 4/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 610us/step - accuracy: 0.7812 - loss: 0.0756 
Epoch 5/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 612ms/step - accuracy: 0.8240 - loss: 0.0675 - val_accuracy: 0.7500 - val_loss: 0.1006


In [9]:
results = pd.DataFrame(model.history.history)

ValueError: All arrays must be of the same length

In [ ]:
results[['loss','val_loss']].plot(title='Model Loss')

In [ ]:
results[['accuracy','val_accuracy']].plot(title='Model Accuracy')

In [ ]:
# Print the accuracy and loss
print("Accuracy: {}, Val_Accuracy: {}".format(results['accuracy'].iloc[-1], results['val_accuracy'].iloc[-1]))
print("Loss: {}, Val_Loss: {}".format(results['loss'].iloc[-1], results['val_loss'].iloc[-1]))

In [ ]:
# collect n batches of images and display the confusion matrix for them.

class_names = ['Cloudy', 'Rain', 'Shine', 'Sunrise']
n = 10 #<--you can change the number of batches selected.

image_batch, classes_batch = next(test_generator)

for batch in range(n):
    temp = next(test_generator)
    image_batch = np.concatenate((image_batch, temp[0]))
    classes_batch = np.concatenate((classes_batch, temp[1]))

classes_batch = classes_batch
y_predict = model.predict(image_batch)

#Confusion matrix for training data
cm = confusion_matrix(y_pred = [np.argmax(x) for x in y_predict], y_true = [np.argmax(x) for x in classes_batch])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
#This function will display a number of random images as well as the predictions for the type of weather.

def predict_one(model, num_images = None):
    image_batch, classes_batch = next(test_generator)
    predicted_batch = model.predict(image_batch)
    for k in range(0,image_batch.shape[0] if num_images is None else num_images):
        image = image_batch[k]
        real_class = class_names[np.argmax(classes_batch[k])]
        predicted_class = class_names[np.argmax(predicted_batch[k])]
        value_predicted = predicted_batch[k]
        isTrue = (real_class == predicted_class)
        plt.figure(k)
        plt.title(str("Correct Prediction" if isTrue else "Incorrect Prediction") + ' - class: ' + real_class + ' - ' + 'predicted: ' + predicted_class + str(value_predicted))
        plt.axis('off')
        plt.imshow(image)

In [ ]:
predict_one(model, 1) #<--you can show multiple pictures with this command. Just fill in the number you want to see.